# Endless Piano (ver. 6.0)

***

## Endless Semi-Generative Performance Piano Music Maker

***

### Powered by tegridy-tools TMIDI & GiantMIDI Dataset https://github.com/bytedance/GiantMIDI-Piano

***

#### Project Los Angeles

#### Tegridy Code 2021

***

# Setup environment

In [ ]:
#@title Install tegridy-tools
!git clone https://github.com/asigalov61/tegridy-tools


In [ ]:
#@title Import all needed modules

print('Loading needed modules. Please wait...')
import os
import copy

from tqdm import auto

import secrets
import random

if not os.path.exists('/content/Dataset'):
    os.makedirs('/content/Dataset')

if not os.path.exists('/content/Output'):
    os.makedirs('/content/Output')

os.chdir('/content/tegridy-tools/tegridy-tools')
import TMIDI

import tqdm
from tqdm import auto

os.chdir('/content/')
print('Loading complete. Enjoy! :)')

# Download and load processed GiantMIDI dataset (Required)

## NOTE: Loading will take about 5 minutes and 10GB RAM

In [ ]:
#@title Download processed GiantMIDI dataset (Required)
# %cd /content/
!wget --no-check-certificate -O GiantMIDI.zip "https://onedrive.live.com/download?cid=8A0D502FC99C608F&resid=8A0D502FC99C608F%2118491&authkey=AKrxNM53z9DGX2Y"
!unzip -j GiantMIDI.zip

In [ ]:
#@title Load the dataset

#@markdown NOTE: This may take a while. Please wait...
slices_length_in_miliseconds = 4000 #@param {type:"slider", min:1000, max:8000, step:1000}
overlap_notes_per_slice = 3 #@param {type:"slider", min:0, max:10, step:1}

print('=' * 50) 
print('Loading GiantMIDI...')
quarter_pairs1 = TMIDI.Tegridy_Any_Pickle_File_Loader('/content/GiantMIDI')
print('Done!')
print('=' * 50)

print('Randomizing the dataset...')
random.shuffle(quarter_pairs1[0])
print('Done!')
print('=' * 50)

print('Slicing the dataset...')
quarter_pairs = []
for qp in auto.tqdm(quarter_pairs1[0]):
  quarter_pairs.extend(TMIDI.Tegridy_Score_Slicer(qp, slices_length_in_miliseconds, overlap_notes=overlap_notes_per_slice)[0])
print('Done!')
print('=' * 50)

print('Generating slices match signatures...')
signatures = []
for qp in auto.tqdm(quarter_pairs):
  x = TMIDI.Tegridy_MIDI_Signature(qp, qp)[1]
  y = [x[2], x[3], x[6], x[7], x[8], x[9]]
  signatures.append(y)
print('Done!')
print('=' * 50)

print('Processing finished! Enjoy! :)')
print('=' * 50)

# (OPTIONAL) Process your own dataset

In [ ]:
#@title Process MIDIs to special MIDI dataset with Tegridy MIDI Processor

desired_dataset_name = "Endless-Piano-Music-Dataset" #@param {type:"string"}
file_name_to_output_dataset_to = "/content/Endless-Piano-Music-Dataset" #@param {type:"string"}
desired_MIDI_channel_to_process = 0 #@param {type:"slider", min:-1, max:16, step:1}
encode_MIDI_channels = False #@param {type:"boolean"}
encode_velocities = False #@param {type:"boolean"}
chordify_input_MIDIs = False #@param {type:"boolean"}
melody_conditioned_encoding = False #@param {type:"boolean"}
melody_pitch_baseline = 60 #@param {type:"slider", min:1, max:127, step:1}
time_denominator = 1 #@param {type:"slider", min:1, max:20, step:1}
chars_encoding_offset = 33 #@param {type:"number"}
slices_length_in_miliseconds = 4000 #@param {type:"slider", min:1000, max:8000, step:1000}

print('TMIDI Processor')
print('Starting up...')

###########

average_note_pitch = 0
min_note = 127
max_note = 0

files_count = 0

ev = 0
notes_list_f = []
chords_list_f = []
melody_list_f = []

chords_list = []
chords_count = 0

melody_chords = []
melody_count = 0

TXT = ''
melody = []
chords = []
bf = 0
###########

print('Loading MIDI files...')
print('This may take a while on a large dataset in particular.')

dataset_addr = "/content/Dataset/"
os.chdir(dataset_addr)
filez = list()
for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    filez += [os.path.join(dirpath, file) for file in filenames]

# Stamping the dataset
print('Stamping the dataset...')

TXT_String = 'DATASET=' + str(desired_dataset_name) + chr(10)
TXT_String += 'CHARS_ENCODING_OFFSET=' + str(chars_encoding_offset) + chr(10)
TXT_String += 'LEGEND=STA-DUR-PTC'
if encode_velocities:
  TXT_String += '-VEL'
if encode_MIDI_channels:
  TXT_String += '-CHA'
TXT_String += chr(10)
pf = []
kar_ev = []
pxp_ev = []
print('Processing MIDI files. Please wait...')
for f in tqdm.auto.tqdm(filez):
  try:
    fn = os.path.basename(f)

    fnn = fn
    fn1 = fnn.split('.')[0]
    fn3 = ['Unknown']

    #fn2 = fn.split('.')[0]
    #fn3 = lakh[str(fn2)]
    #fn1 = fn3[0].split('.')[-2].split('/')[-1]

    TXT, melody, chords = TMIDI.Optimus_MIDI_TXT_Processor(f, 
                                                           line_by_line_output=False, 
                                                           chordify_TXT=chordify_input_MIDIs, 
                                                           output_MIDI_channels=encode_MIDI_channels, 
                                                           char_offset=chars_encoding_offset, 
                                                           dataset_MIDI_events_time_denominator=time_denominator, 
                                                           output_velocity=encode_velocities, 
                                                           MIDI_channel=desired_MIDI_channel_to_process,
                                                           MIDI_patch=range(0,127), 
                                                           melody_conditioned_encoding=melody_conditioned_encoding,
                                                           melody_pitch_baseline=melody_pitch_baseline,
                                                           song_name=fn1, 
                                                           perfect_timings=True)
    chords_list_f.append(chords)

    melody_list_f.append(melody)

    pf.append([fn1, f.split('/')[-2], f.replace('/content/Dataset/','/LAKH/clean_midi/')])


    files_count += 1

  except KeyboardInterrupt:
    print('Exiting...Saving progress...')
    break

  except:
    bf += 1
    print('Bad MIDI:', f)
    print('Count:', bf)
    
    continue

#print('Stamping total number of songs...')
#TXT_String += 'TOTAL_SONGS_COUNT=' + str(files_count)

print('Task complete :)')
print('==================================================')
#print('Number of processed dataset MIDI files:', files_count)
#print('Number of MIDI chords recorded:', len(chords_list_f))
#print('First chord event:', chords_list_f[0], 'Last chord event:', chords_list_f[-1]) 
#print('Number of recorded melody events:', len(melody_list_f))
#print('First melody event:', melody_list_f[0], 'Last Melody event:', melody_list_f[-1])
#print('Total number of MIDI events recorded:', len(chords_list_f) + len(melody_list_f))

# Writing dataset to TXT file
#print('Writing dataset to TXT file...')
#with open(file_name_to_output_dataset_to + '.txt', 'wb') as f:
  #f.write(TXT_String.encode('utf-8', 'replace'))
  #f.close

# Dataset
print('Finalizing the dataset...')
MusicDataset = [chords_list_f, melody_list_f, kar_ev, filez, pf, bf, files_count]

print('Randomizing dataset...')
random.shuffle(chords_list_f)

print('Slicing dataset...')
quarter_pairs = []
for d in auto.tqdm(chords_list_f):
  quarter_pairs.extend(TMIDI.Tegridy_Score_Slicer(d, slices_length_in_miliseconds, overlap_notes=overlap_notes_per_slice)[0])

print('Generating slices match signatures...')
signatures = []
for qp in auto.tqdm(quarter_pairs):
  x = TMIDI.Tegridy_MIDI_Signature(qp, qp)[1]
  y = [x[2], x[3], x[6], x[7], x[8], x[9]]
  signatures.append(y)
print('=' * 50)

print('Done! Enjoy!')
TMIDI.Tegridy_Pickle_File_Writer(MusicDataset, file_name_to_output_dataset_to)

# Generate Endless Classical Piano Music

In [ ]:
#@title Generate Music with the Score Slices Fuzzy Matching

#@markdown NOTE: If nothing is being generated or if the song is too short: re-run the generator.

#@markdown NOTE: If multipliers == 1 performs exact slices match, otherwise the matching is fuzzy based on provided multipliers.

number_of_slices_to_try_to_generate = 10 #@param {type:"slider", min:1, max:20, step:1}

slices_averages_multiplier = 0.1 #@param {type:"slider", min:0, max:1, step:0.05}
slices_sums_multiplier = 0.001 #@param {type:"slider", min:0, max:0.01, step:0.001}
overlap_notes = 3 #@param {type:"slider", min:0, max:10, step:1}

print('=' * 100)
print('Endless Piano')
print('=' * 100)

print('Starting search...')
print('=' * 100)

c = 1

sig = signatures[secrets.randbelow(len(signatures))]

song = []
song.append(quarter_pairs[signatures.index(sig)])

for i in auto.tqdm(range(number_of_slices_to_try_to_generate)):

  for s in signatures:

    div = slices_averages_multiplier
    div2 = slices_sums_multiplier
    s1 = [ int(sig[0] * div2), int(sig[1] * div),int(sig[2] * div2), int(sig[3] * div),int(sig[4] * div2), int(sig[5] * div)]
    s2 = [ int(s[0] * div2), int(s[1] * div),int(s[2] * div2), int(s[3] * div),int(s[4] * div2), int(s[5] * div)]
    if s1 == s2 and signatures.index(s) != signatures.index(sig):
      if quarter_pairs[signatures.index(s)][overlap_notes:] not in song:
        song.append(quarter_pairs[signatures.index(s)][overlap_notes:])
        sig = signatures[signatures.index(s)]
        print('Found', c, 'slices...')
        c += 1
        break
  
  if c == i + 1:
    print('=' * 100)
    print('Generator exhausted. Stopping...')
    break

print('=' * 100)

if c >= i + 1:

  print('Finalizing resulting song...')
  print('=' * 100)
  song1 = []
  for s in song:
    song1.extend(s)
  song1 = [s for s in song1 if type(s) == list]

  print('Analyzing generated song...')
  ptime = 0
  count = 1
  ptime = song1[0][1]
  for s in song1:
    if abs(s[1] - ptime) > 1000:
      count += 1
    ptime = s[1]
  print('Song has', count, 'unique pieces.')
  if count < 2:
    print('PLAGIARIZM WARNING: Your composition is most likely plagiarizm')
  print('=' * 100)

  print('Adding unique pieces labels to the song...')
  song2 = []
  ptime = song1[0][1]
  song2.append(['text_event', song1[0][1], str(song1[0])])
  for s in song1:
    if abs(s[1] - ptime) > 1000:
      song2.append(['text_event', s[1], str(s)])
      song2.append(s)
    else:
      song2.append(s)
    ptime = s[1]
  print('=' * 100)

  print('Recalculating songs timings...')
  song = TMIDI.Tegridy_Timings_Converter(song2)[0]
  print('=' * 100)

  print('Total song length:', len(song))
  print('=' * 100)

  comp_numb = sum([y[4] for y in song if y[0] == 'note'])
  comp_length = len(song)
  print('Endless Piano Composition #:', comp_numb, '-', comp_length)
  print('=' * 100)

  stats = TMIDI.Tegridy_SONG_to_MIDI_Converter(song,
                                              output_signature='Endless Piano',
                                              output_file_name='/content/Endless-Piano-Music-Composition', 
                                              list_of_MIDI_patches=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], 
                                              track_name='Composition #:' + str(comp_numb) + '-' + str(comp_length))
  print('=' * 100)

In [ ]:
#@title Generate Music with the Matching Overlapping Score Slices

#@markdown NOTE: If nothing is being generated or if the song is too short: re-run the generator.

#@markdown NOTE: Overlap notes count should match dataset overlap notes count

number_of_slices_to_try_to_generate = 10 #@param {type:"slider", min:1, max:20, step:1}
overlap_notes = 3 #@param {type:"slider", min:0, max:10, step:1}

print('=' * 100)
print('Endless Piano')
print('=' * 100)

print('Starting search...')
print('=' * 100)

c = 1

song = []
song.append(quarter_pairs[secrets.randbelow(len(quarter_pairs))])

for i in auto.tqdm(range(number_of_slices_to_try_to_generate)):

  for qp in quarter_pairs:

      s1 = [y[2:] for y in song[-1][-overlap_notes:]]
      s2 = [y[2:] for y in qp[:overlap_notes]]
      
      if s1 == s2:
        if qp[overlap_notes:] not in song:
          song.append(qp[overlap_notes:])
          
          print('Found', c, 'slices...')
          c += 1
          break

  if c == i + 1:
    print('=' * 100)
    print('Generator exhausted. Stopping...')
    break

print('=' * 100)

if c >= i + 1:

  print('Finalizing resulting song...')
  print('=' * 100)
  song1 = []
  for s in song:
    song1.extend(s)
  song1 = [s for s in song1 if type(s) == list]

  print('Analyzing generated song...')
  ptime = 0
  count = 1
  ptime = song1[0][1]
  for s in song1:
    if abs(s[1] - ptime) > 1000:
      count += 1
    ptime = s[1]
  print('Song has', count, 'unique pieces.')
  if count < 2:
    print('PLAGIARIZM WARNING: Your composition is most likely plagiarizm')
  print('=' * 100)

  print('Adding unique pieces labels to the song...')
  song2 = []
  ptime = song1[0][1]
  song2.append(['text_event', song1[0][1], str(song1[0])])
  for s in song1:
    if abs(s[1] - ptime) > 1000:
      song2.append(['text_event', s[1], str(s)])
      song2.append(s)
    else:
      song2.append(s)
    ptime = s[1]
  print('=' * 100)

  print('Recalculating songs timings...')
  song = TMIDI.Tegridy_Timings_Converter(song2)[0]
  print('=' * 100)

  print('Total song length:', len(song))
  print('=' * 100)

  comp_numb = sum([y[4] for y in song if y[0] == 'note'])
  comp_length = len(song)
  print('Endless Piano Composition #:', comp_numb, '-', comp_length)
  print('=' * 100)

  stats = TMIDI.Tegridy_SONG_to_MIDI_Converter(song,
                                              output_signature='Endless Piano',
                                              output_file_name='/content/Endless-Piano-Music-Composition', 
                                              list_of_MIDI_patches=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], 
                                              track_name='Composition #:' + str(comp_numb) + '-' + str(comp_length))
  print('=' * 100)

# Plot and Listen

In [ ]:
#@title Install prerequisites
!apt install fluidsynth #Pip does not work for some reason. Only apt works
!pip install midi2audio
!pip install pretty_midi

In [ ]:
#@title Plot and listen to the last generated composition
#@markdown NOTE: May be very slow with the long compositions
from midi2audio import FluidSynth
from IPython.display import display, Javascript, HTML, Audio
import pretty_midi
import librosa.display
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np

print('Synthesizing the last output MIDI. Please stand-by... ')
fname = '/content/Endless-Piano-Music-Composition'

fn = os.path.basename(fname + '.mid')
fn1 = fn.split('.')[0]
print('Playing and plotting composition...')

pm = pretty_midi.PrettyMIDI(fname + '.mid')

# Retrieve piano roll of the MIDI file
piano_roll = pm.get_piano_roll()

plt.figure(figsize=(14, 5))
librosa.display.specshow(piano_roll, x_axis='time', y_axis='cqt_note', sr=64000, cmap=plt.cm.hot)
plt.title('Composition: ' + fn1)

FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname + '.mid'), str(fname + '.wav'))
Audio(str(fname + '.wav'), rate=16000)

# Congrats! You did it! :)